In [21]:
%pip install --upgrade pip 
%pip install pandas pyarrow fastparquet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import json, re, os

In [23]:
data_dir = '../data'

In [24]:
input_file = os.path.join(data_dir, 'raw-data.csv')

In [25]:
print(f"Transformando datos de {input_file}...")
df = pd.read_csv(input_file)

df.columns = [col.strip().lower().replace(' ', '_').replace('.','_') for col in df.columns]

Transformando datos de ../data/raw-data.csv...


# Advertiser

In [26]:
# Separa las tablas
advertiser_cols = [col for col in df.columns if col.startswith("advertiser") or col in ["timestamp", "source"]]
df_advertiser = df[advertiser_cols]

In [27]:
# Declara formatos
custom_funcs = {
    "advertiser_userno": lambda col: col.astype(str),
    "advertiser_monthordercount": lambda col: col.astype(int),
    "advertiser_monthfinishrate": lambda col: col.astype(float),
    "advertiser_positiverate": lambda col: col.astype(float),
    "advertiser_usertype": lambda col: col.astype("category"),
    "advertiser_usergrade": lambda col: col.astype(int),
    "advertiser_useridentity": lambda col: col.astype("category"),
    "advertiser_badges": lambda col: col.astype(str),
    "advertiser_viplevel": lambda col: col.fillna(0).astype(int),
    "advertiser_isblocked": lambda col: col.astype(bool),
    "advertiser_activetimeinsecond": lambda col: col.fillna(-1).astype(int),
    
    "timestamp": lambda col: pd.to_datetime(col, unit="s"),
    "source": lambda col: col.astype("category"),
}

In [28]:
# Pone los formatos
default_func = lambda col: col

for col in df_advertiser.columns:
    df_advertiser.loc[:, col] = custom_funcs.get(col, default_func)(df_advertiser[col])

In [29]:
df[["adv_advno", "advertiser_userno"]].astype(str).groupby("advertiser_userno").agg(','.join)

,adv_advno
advertiser_userno,
s0119d3ae19b832619f554631e41c9d71,12724047536393334784
s027ba514af1233178e8437cf0139a8e8,"13717544266533711872,13730709405136564224,1274..."
s029fc1353366358e806ce573f430e34e,12748816789249544192
s02a6779cfc4c367fb148d6f9c7f5e7d1,12645468749725069312
s03f22b90d96d35aa877cfd22f56b9e18,12734003992719945728
...,...
sfae1f5fb9fa93dacb81e44b23b93bd81,13733187909778223104
sfc54f7fd84f5382ca4d347f2d5fba2bc,12733650529708371968
sfd33765e18ef3fbaa6e06fe3fce61ce8,12696661390019944448


In [30]:
df_advertiser["advices"] = df[["adv_advno", "advertiser_userno"]].astype(str).groupby("advertiser_userno")["adv_advno"].transform(lambda x: ','.join(x))

/tmp/ipykernel_27913/3314834437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_advertiser["advices"] = df[["adv_advno", "advertiser_userno"]].astype(str).groupby("advertiser_userno")["adv_advno"].transform(lambda x: ','.join(x))


In [31]:
df_advertiser.drop_duplicates(subset=["advertiser_userno"], inplace=True)

/tmp/ipykernel_27913/626027465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_advertiser.drop_duplicates(subset=["advertiser_userno"], inplace=True)


In [32]:
df_advertiser.to_parquet(os.path.join(data_dir,'advertiser.parquet'), index=False)
print("advertiser Transformación terminada.")

advertiser Transformación terminada.


# Advice

In [33]:
advice_cols = [col for col in df.columns if (not col.startswith("advertiser") or col in ["advertiser_userno", "advertiser_nickname"]) and not col in []]
df_advice = df[advice_cols]

In [34]:
# Declara formatos
custom_funcs = {
    "adv_advno": lambda col: col.astype(str),
    "adv_classify": lambda col: col.astype("category"),
    "adv_tradetype": lambda col: col.astype("category"),
    "adv_asset": lambda col: col.astype("category"),
    "adv_fiatunit": lambda col: col.astype("category"),
    "adv_price": lambda col: col.astype(float),
    "adv_surplusamount": lambda col: col.astype(float),
    "adv_tradablequantity": lambda col: col.astype(float),
    "adv_maxsingletransamount": lambda col: col.astype(float),
    "adv_minsingletransamount": lambda col: col.astype(float),
    "adv_paytimelimit": lambda col: col.astype(int),
    "adv_takeradditionalkycrequired": lambda col: col.astype(bool),
    "adv_assetscale": lambda col: col.astype(int),
    "adv_fiatscale": lambda col: col.astype(int),
    "adv_pricescale": lambda col: col.astype(int),
    "adv_fiatsymbol": lambda col: col.astype("category"),
    "adv_istradable": lambda col: col.astype(bool),
    "adv_dynamicmaxsingletransamount": lambda col: col.astype(float),
    "adv_minsingletransquantity": lambda col: col.astype(float),
    "adv_maxsingletransquantity": lambda col: col.astype(float),
    "adv_dynamicmaxsingletransquantity": lambda col: col.astype(float),
    "adv_commissionrate": lambda col: col.astype(float),
    "adv_issafepayment": lambda col: col.astype(bool),
    
    "adv_trademethods": lambda col: col.apply(lambda x: [method['identifier'] for method in eval(x)]),
    
    "advertiser_userno": lambda col: col.astype(str),
    "advertiser_nickname": lambda col: col.astype(str),

    "timestamp": lambda col: pd.to_datetime(col, unit="s"),
    "source": lambda col: col.astype("category"),
}

In [35]:
# Pone los formatos
default_func = lambda col: col

for col in df_advice.columns:
    df_advice.loc[:, col] = custom_funcs.get(col, default_func)(df_advice[col])

In [36]:
df_advice.to_parquet(os.path.join(data_dir,'advice.parquet'), index=False)
print("advice Transformación terminada.")

advice Transformación terminada.


# Trade Methods

In [37]:
trade_methods_cols = [
    col
    for col in df.columns
    if (col in ["adv_advno", "adv_trademethods"]) and not col in []
]
df_advice_trade_info = df[trade_methods_cols]

In [ ]:
df_advice_trade_table = df_advice_trade_info["adv_trademethods"].apply(eval).explode()

df_trade_methods_table = pd.json_normalize(df_advice_trade_table)

df_trade_methods_table["adv_advno"] = df_advice_trade_info.loc[df_advice_trade_table.index, "adv_advno"].values

df_trade_methods_table = df_trade_methods_table.drop_duplicates()

df_trade_methods = df_trade_methods_table.groupby("identifier").agg({
    "adv_advno": set,
    "tradeMethodName": set,
    "tradeMethodShortName": set,
    "tradeMethodBgColor": set
}).reset_index()

df_trade_methods = df_trade_methods.applymap(lambda x: {str(i) for i in x if i is not None} if isinstance(x, set) else x)
df_trade_methods = df_trade_methods.applymap(lambda x: ','.join(x) if isinstance(x, set) else x)

In [42]:
df_trade_methods.to_parquet(os.path.join(data_dir,'trade_methods.parquet'), index=False)
print("trade_methods Transformación terminada.")

trade_methods Transformación terminada.
